In [3]:
import os
import re
import glob
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree

In [19]:
MicMSE = np.load("/home/77462217B/lois/AAADMeth/data/datasets/cases/Mic10Cas_float16.npy").T
pdMic=pd.read_csv("/mnt/hydra/ubs/shared/users/Lois/DatasetsControles/OnlyMichaud/pdMichaud.txt", sep=",")

In [20]:
np.save("/home/77462217B/lois/AAADMeth/data/datasets/cases/Mic10Cas_float16.npy",MicMSE)

In [22]:
keep = (pdMic["Status"]==0)
keep = keep.to_numpy()
MicConMSE = pd.DataFrame(MicMSE).iloc[keep,:]
pdMicCon = pdMic.iloc[keep,:]

keep = (pdMic["Status"]==1)
keep = keep.to_numpy()
MicCasMSE = pd.DataFrame(MicMSE).iloc[keep,:]
pdMicCas = pdMic.iloc[keep,:]

keep = (pdMicCas["YearsToDiagnosis"]<=5)
keep = keep.to_numpy()
Mic5CasMSE = MicCasMSE.iloc[keep,:]

keep = (pdMicCas["YearsToDiagnosis"]<=3) 
keep = keep.to_numpy()
Mic3CasMSE = MicCasMSE.iloc[keep,:]

keep = (pdMicCas["YearsToDiagnosis"]>10)
keep = keep.to_numpy()
MicM10CasMSE = MicCasMSE.iloc[keep,:]

keep = (pdMicCas["YearsToDiagnosis"]<=10) & (pdMicCas["YearsToDiagnosis"]>5)
keep = keep.to_numpy()
Mic10CasMSE = MicCasMSE.iloc[keep,:]


In [25]:
np.save("/home/77462217B/lois/AAADMeth/data/datasets/cases/Mic3Cas_float16.npy", Mic3CasMSE)

In [ ]:
def group_columns_by_mean(data, group_size: int = 10, missing_value: float = -1):
    
    if isinstance(data, np.ndarray):
        df = pd.DataFrame(data)
    elif isinstance(data, pd.DataFrame):
        df = data
    else:
        raise TypeError("Input should be a pandas.DataFrame or a numpy.ndarray.")

    n_cols = df.shape[1]
    grouped_cols = []

    for start in range(0, n_cols, group_size):
        end = min(start + group_size, n_cols)
        subset = df.iloc[:, start:end]
        group_mean = subset.mask(subset.eq(missing_value), np.nan).mean(axis=1, skipna=True)
        group_mean = group_mean.fillna(missing_value)
        grouped_cols.append(group_mean)

    grouped_matrix = pd.concat(grouped_cols, axis=1)
    return grouped_matrix

In [4]:
MicMSE = group_columns_by_mean(MicMSE, group_size=10)
TrainConMSE = group_columns_by_mean(TrainConMSE, group_size=10)

In [6]:
keep = (pdMic["Cohort"]=="PHS")
keep = keep.to_numpy()
EVMicMSE = MicMSE.iloc[keep,:]
EVpdMic = pdMic.iloc[keep,:]

keep = (pdMic["Cohort"]!="PHS")
keep = keep.to_numpy()
CVMicMSE = MicMSE.iloc[keep,:]
CVpdMic = pdMic.iloc[keep,:]

keep = (CVpdMic["Status"]==0)
keep = keep.to_numpy()
CVMicConMSE = CVMicMSE.iloc[keep,:]
CVpdMicCon = CVpdMic.iloc[keep,:]

keep = (CVpdMic["Status"]==1)
keep = keep.to_numpy()
CVMicCasMSE = CVMicMSE.iloc[keep,:]
CVpdMicCas = CVpdMic.iloc[keep,:]

keep = (EVpdMic["Status"]==0)
keep = keep.to_numpy()
EVMicConMSE = EVMicMSE.iloc[keep,:]
EVpdMicCon = EVpdMic.iloc[keep,:]

keep = (EVpdMic["Status"]==1)
keep = keep.to_numpy()
EVMicCasMSE = EVMicMSE.iloc[keep,:]
EVpdMicCas = EVpdMic.iloc[keep,:]

keep = (CVpdMicCas["YearsToDiagnosis"]<=5)
keep = keep.to_numpy()
CVMic5CasMSE = CVMicCasMSE.iloc[keep,:]

keep = (CVpdMicCas["YearsToDiagnosis"]<=3) 
keep = keep.to_numpy()
CVMic3CasMSE = CVMicCasMSE.iloc[keep,:]

keep = (CVpdMicCas["YearsToDiagnosis"]>10)
keep = keep.to_numpy()
CVMicM10CasMSE = CVMicCasMSE.iloc[keep,:]

keep = (CVpdMicCas["YearsToDiagnosis"]<=10) & (CVpdMicCas["YearsToDiagnosis"]>5)
keep = keep.to_numpy()
CVMic10CasMSE = CVMicCasMSE.iloc[keep,:]

keep = (EVpdMicCas["YearsToDiagnosis"]<=5)
keep = keep.to_numpy()
EVMic5CasMSE = EVMicCasMSE.iloc[keep,:]

keep = (EVpdMicCas["YearsToDiagnosis"]<=3) 
keep = keep.to_numpy()
EVMic3CasMSE = EVMicCasMSE.iloc[keep,:]

keep = (EVpdMicCas["YearsToDiagnosis"]>10)
keep = keep.to_numpy()
EVMicM10CasMSE = EVMicCasMSE.iloc[keep,:]

keep = (EVpdMicCas["YearsToDiagnosis"]<=10) & (EVpdMicCas["YearsToDiagnosis"]>5)
keep = keep.to_numpy()
EVMic10CasMSE = EVMicCasMSE.iloc[keep,:]

MSE Analysis

In [8]:
def BackgroundNormalization(ErrorsMatrix):
    BCErrorsMatrix={}
    for i in range(ErrorsMatrix.shape[0]):
        #BCErrorsMatrix[i] = ErrorsMatrix.loc[i,:]/(ErrorsMatrix.loc[i,:].mean())
        BCErrorsMatrix[i] = ErrorsMatrix.iloc[i,:]/(ErrorsMatrix.iloc[i,:].median())
    BCErrorsMatrix = pd.DataFrame(BCErrorsMatrix).T
    return BCErrorsMatrix

In [9]:
BCTrainConMSE =  BackgroundNormalization(TrainConMSE)

EVMicMSE = BackgroundNormalization(EVMicMSE)
EVMicConMSE = BackgroundNormalization(EVMicConMSE)
EVMicCasMSE = BackgroundNormalization(EVMicCasMSE)
EVMic5CasMSE = BackgroundNormalization(EVMic5CasMSE)
EVMic3CasMSE = BackgroundNormalization(EVMic3CasMSE)
EVMicM10CasMSE = BackgroundNormalization(EVMicM10CasMSE)
EVMic10CasMSE = BackgroundNormalization(EVMic10CasMSE)

CVMicMSE = BackgroundNormalization(CVMicMSE)
CVMicConMSE = BackgroundNormalization(CVMicConMSE)
CVMicCasMSE = BackgroundNormalization(CVMicCasMSE)
CVMic5CasMSE = BackgroundNormalization(CVMic5CasMSE)
CVMic3CasMSE = BackgroundNormalization(CVMic3CasMSE)
CVMicM10CasMSE = BackgroundNormalization(CVMicM10CasMSE)
CVMic10CasMSE = BackgroundNormalization(CVMic10CasMSE)

In [10]:
def ScoreRegion(BCErrorsMatrix, BCErrorsMatrixControls):
    ScoreMatrix=((BCErrorsMatrix-BCErrorsMatrixControls.mean(axis=0))/(BCErrorsMatrixControls.std(axis=0)))
    return ScoreMatrix

In [13]:
EVMicMSE = ScoreRegion(EVMicMSE, BCTrainConMSE)
EVMicConMSE = ScoreRegion(EVMicConMSE, BCTrainConMSE)
EVMicCasMSE = ScoreRegion(EVMicCasMSE, BCTrainConMSE)
EVMic5CasMSE = ScoreRegion(EVMic5CasMSE, BCTrainConMSE)
EVMic3CasMSE = ScoreRegion(EVMic3CasMSE, BCTrainConMSE)
EVMicM10CasMSE = ScoreRegion(EVMicM10CasMSE, BCTrainConMSE)
EVMic10CasMSE = ScoreRegion(EVMic10CasMSE, BCTrainConMSE)

CVMicMSE = ScoreRegion(CVMicMSE, BCTrainConMSE)
CVMicConMSE = ScoreRegion(CVMicConMSE, BCTrainConMSE)
CVMicCasMSE = ScoreRegion(CVMicCasMSE, BCTrainConMSE)
CVMic5CasMSE = ScoreRegion(CVMic5CasMSE, BCTrainConMSE)
CVMic3CasMSE = ScoreRegion(CVMic3CasMSE, BCTrainConMSE)
CVMicM10CasMSE = ScoreRegion(CVMicM10CasMSE, BCTrainConMSE)
CVMic10CasMSE = ScoreRegion(CVMic10CasMSE, BCTrainConMSE)

/home/77462217B/.local/lib/python3.12/site-packages/numpy/_core/_methods.py:52: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


In [12]:
EVMicMSE.to_csv('/home/77462217B/lois/ADMeth/outcomes/MRS/EV/EVMicMSE.csv', index=False)
EVMicConMSE.to_csv('/home/77462217B/lois/ADMeth/outcomes/MRS/EV/EVMicConMSE.csv', index=False)
EVMicCasMSE.to_csv('/home/77462217B/lois/ADMeth/outcomes/MRS/EV/EVMicCasMSE.csv', index=False)
EVMic5CasMSE.to_csv('/home/77462217B/lois/ADMeth/outcomes/MRS/EV/EVMic5CasMSE.csv', index=False)
EVMic3CasMSE.to_csv('/home/77462217B/lois/ADMeth/outcomes/MRS/EV/EVMic3CasMSE.csv', index=False)
EVMicM10CasMSE.to_csv('/home/77462217B/lois/ADMeth/outcomes/MRS/EV/EVMicM10CasMSE.csv', index=False)
EVMic10CasMSE.to_csv('/home/77462217B/lois/ADMeth/outcomes/MRS/EV/EVMic10CasMSE.csv', index=False)

CVMicMSE.to_csv('/home/77462217B/lois/ADMeth/outcomes/MRS/CV/CVMicMSE.csv', index=False)
CVMicConMSE.to_csv('/home/77462217B/lois/ADMeth/outcomes/MRS/CV/CVMicConMSE.csv', index=False)
CVMicCasMSE.to_csv('/home/77462217B/lois/ADMeth/outcomes/MRS/CV/CVMicCasMSE.csv', index=False)
CVMic5CasMSE.to_csv('/home/77462217B/lois/ADMeth/outcomes/MRS/CV/CVMic5CasMSE.csv', index=False)
CVMic3CasMSE.to_csv('/home/77462217B/lois/ADMeth/outcomes/MRS/CV/CVMic3CasMSE.csv', index=False)
CVMicM10CasMSE.to_csv('/home/77462217B/lois/ADMeth/outcomes/MRS/CV/CVMicM10CasMSE.csv', index=False)
CVMic10CasMSE.to_csv('/home/77462217B/lois/ADMeth/outcomes/MRS/CV/CVMic10CasMSE.csv', index=False)

In [ ]:
def DiffControls(BCErrorsMatrix, BCErrorsMatrixControls):
    DiffErrorsMatrix=((BCErrorsMatrix.mean(axis=0)-BCErrorsMatrixControls.mean(axis=0))/BCErrorsMatrixControls.mean(axis=0))
    return DiffErrorsMatrix

In [ ]:
DiffMicConMSE = DiffControls(BCMicConMSE, BCMicConMSE)
DiffMicCasMSE = DiffControls(BCMicCasMSE, BCMicConMSE)
DiffMic5CasMSE = DiffControls(BCMic5CasMSE, BCMicConMSE)
DiffMic3CasMSE = DiffControls(BCMic3CasMSE, BCMicConMSE)
DiffMic10CasMSE = DiffControls(BCMic10CasMSE, BCMicConMSE)
DiffMicM10CasMSE = DiffControls(BCMicM10CasMSE, BCMicConMSE)

In [ ]:

Mic3ORI = pd.read_csv("/home/77462217B/lois/AAImageneAnomalyDetection/outcomes/REC/heavymodelv1/All/StdPlus1ScoreMic3CasMSE.csv")

In [2]:
import pandas as pd 
import numpy as np

Mic3ORI = pd.read_csv("/home/77462217B/lois/AAImageneAnomalyDetection/outcomes/REC/heavymodelv1/All/StdPlus1ScoreMic3CasMSE.csv")

In [3]:
Mic3ORI

,0,1,2,3,4,5,6,7,8,9,...,319990,319991,319992,319993,319994,319995,319996,319997,319998,319999
0,-0.174441,-0.154864,0.723324,-0.005013,-0.048955,-0.092836,0.669281,8.163758,-0.108669,-0.144102,...,-0.116183,-0.064590,-0.116263,-0.144609,-0.066398,0.078720,-0.070829,-0.107034,-0.104006,-0.089889
1,-0.157920,-0.109892,0.524336,0.002256,0.009985,-0.083765,0.678565,0.063426,-0.107945,-0.144228,...,-0.115883,-0.012477,-0.117067,-0.107564,0.045421,-0.075992,-0.083703,-0.132107,-0.104626,-0.089994
2,-0.164490,-0.157421,0.375013,0.073205,-0.112031,-0.041822,0.344867,0.692974,-0.108526,-0.144304,...,-0.115408,-0.012328,-0.055939,-0.141234,-0.062557,-0.031412,-0.060282,-0.106895,-0.104589,-0.089894
3,0.534804,-0.155223,-0.120842,-0.090940,0.145328,-0.065083,-0.142321,-0.340014,-0.110584,-0.144607,...,-0.115388,-0.096523,-0.043205,-0.125252,-0.084453,-0.045536,-0.042341,-0.113867,-0.103899,-0.089793
4,-0.019509,-0.132558,3.243140,-0.059241,0.427454,-0.069499,0.465446,3.772467,-0.108488,-0.144375,...,-0.115330,-0.080066,-0.057994,-0.128577,-0.080826,0.025508,-0.092841,-0.132117,-0.103979,-0.089987
5,-0.159298,-0.137831,0.269907,-0.083163,0.327821,0.065057,0.450626,1.367274,-0.108362,-0.144458,...,-0.114491,-0.095561,-0.095970,-0.059837,-0.084419,0.022767,-0.092714,-0.133379,-0.104585,-0.089746
6,0.136206,-0.034192,-0.117659,-0.079606,0.359796,0.245083,-0.136986,-0.414147,-0.110036,6.890980,...,-0.115421,-0.096244,0.030046,-0.141031,-0.085873,-0.060292,-0.091094,-0.111088,-0.102102,-0.089816
7,0.066547,-0.157007,0.856471,-0.111722,-0.109589,-0.095451,0.602658,1.613682,-0.109240,-0.144387,...,-0.116118,-0.056087,0.222569,-0.137617,20.120274,0.026377,-0.092021,-0.110541,-0.103547,-0.089993
8,-0.163705,-0.142767,0.331003,0.231692,-0.024267,-0.082072,0.776322,0.103209,-0.108597,-0.144042,...,-0.114615,-0.055827,0.001361,-0.137813,-0.087676,-0.056786,-0.092734,-0.109170,-0.101663,-0.089899
9,-0.012262,-0.157817,-0.065600,0.106182,-0.094228,-0.088367,0.040745,-0.374221,-0.108762,-0.144142,...,-0.115898,-0.089400,-0.093144,-0.125807,-0.087896,-0.079381,-0.093673,-0.131662,-0.103684,-0.089617


In [28]:
Mic3 = pd.DataFrame(np.load("/home/77462217B/lois/AAADMeth/data/recscores/Michaud_v1_3cases/cases/Mic3Cas_float16_scores.npy"))

In [29]:
Mic3

,0,1,2,3,4,5,6,7,8,9,...,319990,319991,319992,319993,319994,319995,319996,319997,319998,319999
0,-0.174441,-0.154864,0.723324,-0.005013,-0.048955,-0.092836,0.669281,8.163758,-0.108669,-0.144102,...,-0.116183,-0.064590,-0.116263,-0.144609,-0.066398,0.078720,-0.070829,-0.107034,-0.104006,-0.089889
1,-0.157920,-0.109892,0.524336,0.002256,0.009985,-0.083765,0.678565,0.063426,-0.107945,-0.144228,...,-0.115883,-0.012477,-0.117067,-0.107564,0.045421,-0.075992,-0.083703,-0.132107,-0.104626,-0.089994
2,-0.164490,-0.157421,0.375013,0.073205,-0.112031,-0.041822,0.344867,0.692974,-0.108526,-0.144304,...,-0.115408,-0.012328,-0.055939,-0.141234,-0.062557,-0.031412,-0.060282,-0.106895,-0.104589,-0.089894
3,0.534804,-0.155223,-0.120842,-0.090940,0.145328,-0.065083,-0.142321,-0.340014,-0.110584,-0.144607,...,-0.115388,-0.096523,-0.043205,-0.125252,-0.084453,-0.045536,-0.042341,-0.113867,-0.103899,-0.089793
4,-0.019509,-0.132558,3.243140,-0.059241,0.427454,-0.069499,0.465446,3.772467,-0.108488,-0.144375,...,-0.115330,-0.080066,-0.057994,-0.128577,-0.080826,0.025508,-0.092841,-0.132117,-0.103979,-0.089987
5,-0.159298,-0.137831,0.269907,-0.083163,0.327821,0.065057,0.450626,1.367274,-0.108362,-0.144458,...,-0.114491,-0.095561,-0.095970,-0.059837,-0.084419,0.022767,-0.092714,-0.133379,-0.104585,-0.089746
6,0.136206,-0.034192,-0.117659,-0.079606,0.359796,0.245083,-0.136986,-0.414147,-0.110036,6.890980,...,-0.115421,-0.096244,0.030046,-0.141031,-0.085873,-0.060292,-0.091094,-0.111088,-0.102102,-0.089816
7,0.066547,-0.157007,0.856471,-0.111722,-0.109589,-0.095451,0.602658,1.613682,-0.109240,-0.144387,...,-0.116118,-0.056087,0.222569,-0.137617,20.120274,0.026377,-0.092021,-0.110541,-0.103547,-0.089993
8,-0.163705,-0.142767,0.331003,0.231692,-0.024267,-0.082072,0.776322,0.103209,-0.108597,-0.144042,...,-0.114615,-0.055827,0.001361,-0.137813,-0.087676,-0.056786,-0.092734,-0.109170,-0.101663,-0.089899
9,-0.012262,-0.157817,-0.065600,0.106182,-0.094228,-0.088367,0.040745,-0.374221,-0.108762,-0.144142,...,-0.115898,-0.089400,-0.093144,-0.125807,-0.087896,-0.079381,-0.093673,-0.131662,-0.103684,-0.089617


In [ ]:
MicMSE= pd.DataFrame(np.load("/home/77462217B/lois/AAImageneAnomalyDetection/outcomes/MSE/heavymodelv1/Michaud_float16/Michaud_float16_mse_per_sample_per_position.npy"))
pdMic=pd.read_csv("/mnt/hydra/ubs/shared/users/Lois/DatasetsControles/OnlyMichaud/pdMichaud.txt", sep=",")

keep = (pdMic["Status"]==1)
keep = keep.to_numpy()
MicCasMSE = MicMSE.iloc[keep,:]
pdMicCas = pdMic.iloc[keep,:]
keep = (pdMicCas["YearsToDiagnosis"]<=3) 
keep = keep.to_numpy()
Mic3CasMSEORI = MicCasMSE.iloc[keep,:]

Mic3CasMSEORI